In [1]:
from datetime import datetime
import sys

sys.path.append('../embeddings/')
sys.path.append('./embeddings/')
sys.path.append('..')

from dataset import ImageDataLoader
from torch.utils.data import DataLoader
from utils import slice_image_paths
import matplotlib.pyplot as plt
import pickle
import torch
import torchvision
from efficientnet_pytorch import EfficientNet
import numpy as np

from lightly import loss

from lightly.data import LightlyDataset

from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SwaV(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(1280, 1280, 128)
        self.prototypes = SwaVPrototypes(128, n_prototypes=1280)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        x = self.projection_head(x)
        x = torch.nn.functional.normalize(x, dim=1, p=2)
        p = self.prototypes(x)
        return p


In [2]:
caminho_do_arquivo = '../notebooks/efficientnet_SwaV_model.pth'
model = torch.load(caminho_do_arquivo)





model.to(device)
model.eval()


SwaV(
  (backbone): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
    

In [3]:

data = ImageDataLoader('../data/02_data_split/test_data/')
dataloader = DataLoader(data.dataset, batch_size=50, shuffle=False)

target = []
paths = []
labels = []
feature_embeddings = np.empty((0, 1280))



for i, (x, y, path, label) in enumerate(dataloader):
    x = x.to(device=device)
    with torch.no_grad():
        batch_features = model(x)

    batch_features_np = batch_features.view(batch_features.size(0), -1).cpu().numpy()
    feature_embeddings = np.vstack((feature_embeddings, batch_features_np))
    target.extend(list(y.cpu().detach().numpy()))
    paths.extend(slice_image_paths(path))
    labels.extend(label)


data_dict = {
    "model": 'efficientnet_SwaV',
    "embedding":feature_embeddings,
    "target":target,
    "paths": paths,
    "classes":labels
}



with open(f'./efficientnet_SwaV_test.pickle', 'wb') as pickle_file:
    pickle.dump(data_dict, pickle_file)